<a href="https://colab.research.google.com/github/jamessutton600613-png/GC/blob/main/Untitled200.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === GQR TDSE — Minimal HC vs HC+B(-0.6) run with composite movie (fast test) ===
# Outputs (in ./gqr_min_fig):
#   - A_HC and A_HC_B-0p60 folders with per-frame PNGs and MP4s
#   - Composite side-by-side MP4: composite_HC_vs_HCB.mp4
#   - Two-panel PNG snapshot: Fig_mini_HC_vs_HCB.png
# Works on CPU; uses CuPy (GPU) if available.

import os, math, csv, sys, glob
from dataclasses import dataclass

# --- optional GPU backend ---
USE_GPU = False
try:
    import cupy as cp
    _ = cp.zeros((1,))
    USE_GPU = True
except Exception:
    cp = None

import numpy as np
xp = cp if USE_GPU else np

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import imageio.v2 as iio

print(f"[backend] {'CuPy (GPU)' if USE_GPU else 'NumPy (CPU)'}")

# --- quick switches ---
FAST_TEST = True   # set False later for higher fidelity
# fast test settings
NX_NY      = 160 if FAST_TEST else 320
LBOX       = 18.0 if FAST_TEST else 24.0
T_FINAL    = 200.0 if FAST_TEST else 24.0
DT         = 0.012
SAVE_EVERY = 10    # frame every ~0.12 time units

# ---------------- utils ----------------
def ensure_dir(p): os.makedirs(p, exist_ok=True)

def to_cpu(a):
    if USE_GPU and isinstance(a, cp.ndarray):
        return cp.asnumpy(a)
    return np.asarray(a)

def pad_to_mod16(img: Image.Image, bg=(0,0,0)):
    """Pad PIL image to width/height divisible by 16 (codec friendly)."""
    w, h = img.size
    W = (w + 15) // 16 * 16
    H = (h + 15) // 16 * 16
    if (W,H) == (w,h):
        return img
    canvas = Image.new("RGB", (W,H), bg)
    canvas.paste(img, (0,0))
    return canvas

# ---------------- grid & physics ----------------
@dataclass
class Grid:
    nx: int = NX_NY
    ny: int = NX_NY
    Lx: float = LBOX
    Ly: float = LBOX
    hbar: float = 1.0
    m: float = 1.0

    def build(self):
        x = xp.linspace(-self.Lx/2, self.Lx/2, self.nx, endpoint=False)
        y = xp.linspace(-self.Ly/2, self.Ly/2, self.ny, endpoint=False)
        dx = x[1]-x[0]; dy = y[1]-y[0]
        kx = 2*xp.pi*(xp.fft.fftfreq(self.nx, d=float(dx)))
        ky = 2*xp.pi*(xp.fft.fftfreq(self.ny, d=float(dy)))
        KX, KY = xp.meshgrid(kx, ky, indexing='xy')
        X,  Y  = xp.meshgrid(x,  y,  indexing='xy')
        K2 = KX**2 + KY**2
        return X, Y, K2, dx, dy

# --- potentials: A baseline + H (funnel) + C (field) + B (tilt, signed) ---
def potential_A(X, Y, A_height=8.0, A_width=1.3):
    Vx = A_height * xp.exp(-(X/A_width)**2)
    Vy = 0.015 * Y**2
    return Vx + Vy

def gate_H_funnel(X, Y, H_depth=4.0, aperture_y=0.0, aperture_x0=-1.0):
    taper = 0.5 + 0.5*xp.tanh((X - aperture_x0)/2.0)     # 0→1 left→right
    sigma = 0.8 + (1.8 - 0.8)*(1.0 - taper)              # narrows to right
    d = (Y - aperture_y)/sigma
    return -H_depth * xp.exp(-0.5 * d**2) * (0.2 + 0.8*taper)

def gate_C_field(X, Y, C_strength=0.6, theta_deg=0.0):
    th = xp.deg2rad(theta_deg)
    Ex, Ey = C_strength*xp.cos(th), C_strength*xp.sin(th)
    return -(Ex*X + Ey*Y)

def gate_B_tilt(X, Y, B_strength_signed=-0.6):
    # B acts mostly along y; small x component to avoid strict symmetry
    return -B_strength_signed * (0.25*X + 1.0*Y)

def build_V(flags, X, Y,
            A=dict(A_height=8.0, A_width=1.3),
            H=dict(H_depth=4.0, aperture_y=0.0, aperture_x0=-1.0),
            C=dict(C_strength=0.6, theta_deg=0.0),
            B=None):
    V = potential_A(X, Y, **A)
    if flags.get('H', False): V += gate_H_funnel(X, Y, **H)
    if flags.get('C', False): V += gate_C_field(X, Y, **C)
    if flags.get('B', False):
        B_strength = flags.get('B_strength', -0.6)  # signed
        V += gate_B_tilt(X, Y, B_strength_signed=B_strength)
    return V

# --- initial packet ---
def gaussian_packet(X, Y, x0=-6.2, y0=0.0, px=2.1, py=0.0, sx=1.2, sy=1.2, hbar=1.0):
    phase = (px*X + py*Y)/hbar
    amp   = xp.exp(-0.5*((X-x0)/sx)**2 - 0.5*((Y-y0)/sy)**2)
    psi0  = amp * xp.exp(1j*phase)
    psi0 /= xp.sqrt(xp.sum(xp.abs(psi0)**2))
    return psi0

# --- propagation & rendering ---
def run_movie(tag, flags, out_root, vmax=None, fps=12):
    cond_dir = os.path.join(out_root, tag); ensure_dir(cond_dir)
    G = Grid()
    X, Y, K2, dx, dy = G.build()
    hbar, m = G.hbar, G.m

    V = build_V(flags, X, Y)

    # kinetic phase (split-operator)
    Kphase = xp.exp(-1j*(hbar**2*K2)*DT/(2.0*m*hbar))

    # absorber (cosine walls)
    ax = int(0.08*G.nx); ay = int(0.08*G.ny)
    wx = xp.ones(G.nx); wy = xp.ones(G.ny)
    if ax>0:
        r = xp.linspace(0,1,ax)
        wx[:ax]  = xp.cos(0.5*xp.pi*(1-r))**4
        wx[-ax:] = xp.cos(0.5*xp.pi*(r))**4
    if ay>0:
        r = xp.linspace(0,1,ay)
        wy[:ay]  = xp.cos(0.5*xp.pi*(1-r))**4
        wy[-ay:] = xp.cos(0.5*xp.pi*(r))**4
    ABS = wy[:,None]*wx[None,:]

    psi = gaussian_packet(X, Y)

    nsteps = int(T_FINAL/DT)
    frame_paths = []
    extent = [float(to_cpu(X).min()), float(to_cpu(X).max()),
              float(to_cpu(Y).min()), float(to_cpu(Y).max())]

    # choose v-max from early frame if not set
    dens0 = to_cpu(xp.abs(psi)**2)
    if vmax is None:
        vmax = max(np.percentile(dens0, 99.5), 1e-8)

    # per-frame PNGs
    for n in range(nsteps+1):
        if n % SAVE_EVERY == 0 or n == nsteps:
            dens = to_cpu(xp.abs(psi)**2)
            fig, ax = plt.subplots(figsize=(4.6,3.6), dpi=140)
            im = ax.imshow(dens.T, origin='lower', extent=extent, vmin=0, vmax=vmax, aspect='auto')
            ax.set_title(tag.replace('_',' + ') + f"   t={n*DT:.2f}")
            ax.set_xlabel("x"); ax.set_ylabel("y")
            fig.colorbar(im, ax=ax, shrink=0.8)
            fp = os.path.join(cond_dir, f"{tag}_frame_{n:04d}.png")
            fig.savefig(fp, bbox_inches='tight'); plt.close(fig)
            frame_paths.append(fp)

        # split-operator step
        psi *= xp.exp(-1j*V*DT/(2*hbar))
        psi_k = xp.fft.fft2(psi)
        psi_k *= Kphase
        psi   = xp.fft.ifft2(psi_k)
        psi *= xp.exp(-1j*V*DT/(2*hbar))
        psi *= ABS

    # write MP4
    mp4_path = os.path.join(cond_dir, f"{tag}.mp4")
    with iio.get_writer(mp4_path, fps=fps, codec='libx264', quality=8) as w:
        for p in frame_paths:
            im = Image.open(p).convert("RGB")
            im = pad_to_mod16(im)
            w.append_data(np.asarray(im))
    print(f"[{tag}] wrote", mp4_path)

    return frame_paths, mp4_path

# ---------------- run two conditions ----------------
out_root = "gqr_min_fig"; ensure_dir(out_root)

conditions = [
    ("A_HC",          dict(H=True, C=True,  B=False)),
    ("A_HC_B-0p60",   dict(H=True, C=True,  B=True, B_strength=-0.6)),
]

all_frame_lists = {}
all_mp4s = {}
for tag, flags in conditions:
    frames, mp4 = run_movie(tag, flags, out_root)
    all_frame_lists[tag] = frames
    all_mp4s[tag] = mp4

# ---------------- composite side-by-side MP4 ----------------
# align on the smallest number of frames
min_frames = min(len(v) for v in all_frame_lists.values())
left_tag, right_tag = conditions[0][0], conditions[1][0]
left_frames  = all_frame_lists[left_tag][:min_frames]
right_frames = all_frame_lists[right_tag][:min_frames]

# use first pair to set canvas
L0 = Image.open(left_frames[0]).convert("RGB")
R0 = Image.open(right_frames[0]).convert("RGB")
tile_h = max(L0.height, R0.height)
tile_w = max(L0.width,  R0.width)
GAP    = 16
label_h = 36

# simple label drawer (fallback font)
def annotate(img: Image.Image, text: str, pos=(10,5)):
    draw = ImageDraw.Draw(img)
    # white text with black shadow for readability
    draw.text((pos[0]+1, pos[1]+1), text, fill=(0,0,0))
    draw.text(pos, text, fill=(255,255,255))
    return img

composite_path = os.path.join(out_root, "composite_HC_vs_HCB.mp4")
with iio.get_writer(composite_path, fps=12, codec="libx264", quality=9) as w:
    for lf, rf in zip(left_frames, right_frames):
        L = Image.open(lf).convert("RGB").resize((tile_w, tile_h), Image.BILINEAR)
        R = Image.open(rf).convert("RGB").resize((tile_w, tile_h), Image.BILINEAR)
        W = tile_w*2 + GAP
        H = tile_h + label_h
        canvas = Image.new("RGB", (W, H), (10,10,10))
        canvas.paste(L, (0, label_h))
        canvas.paste(R, (tile_w+GAP, label_h))
        annotate(canvas, left_tag.replace('_',' + '),  (10, 6))
        annotate(canvas, right_tag.replace('_',' + '), (tile_w+GAP+10, 6))
        canvas = pad_to_mod16(canvas)
        w.append_data(np.asarray(canvas))

print("[composite] wrote", composite_path)

# ---------------- small 2-panel PNG snapshot (mid frame) ----------------
mid_idx = min_frames//2
snap_L = Image.open(left_frames[mid_idx]).convert("RGB")
snap_R = Image.open(right_frames[mid_idx]).convert("RGB")
tile_h = max(snap_L.height, snap_R.height)
tile_w = max(snap_L.width,  snap_R.width)
GAP    = 12
W = tile_w*2 + GAP
H = tile_h + label_h
panel = Image.new("RGB", (W,H), (255,255,255))
panel.paste(snap_L.resize((tile_w, tile_h), Image.BILINEAR), (0, label_h))
panel.paste(snap_R.resize((tile_w, tile_h), Image.BILINEAR), (tile_w+GAP, label_h))
draw = ImageDraw.Draw(panel)
draw.text((10,6), left_tag.replace('_',' + '),  fill=(0,0,0))
draw.text((tile_w+GAP+10,6), right_tag.replace('_',' + '), fill=(0,0,0))
panel_path = os.path.join(out_root, "Fig_mini_HC_vs_HCB.png")
panel.save(panel_path, "PNG")

print("\n[done]")
print("Folders:", out_root)
print(" - Control MP4:", all_mp4s[left_tag])
print(" - Tilt MP4:   ", all_mp4s[right_tag])
print(" - Composite:  ", composite_path)
print(" - 2-panel PNG:", panel_path)

[backend] CuPy (GPU)
[A_HC] wrote gqr_min_fig/A_HC/A_HC.mp4
[A_HC_B-0p60] wrote gqr_min_fig/A_HC_B-0p60/A_HC_B-0p60.mp4
[composite] wrote gqr_min_fig/composite_HC_vs_HCB.mp4

[done]
Folders: gqr_min_fig
 - Control MP4: gqr_min_fig/A_HC/A_HC.mp4
 - Tilt MP4:    gqr_min_fig/A_HC_B-0p60/A_HC_B-0p60.mp4
 - Composite:   gqr_min_fig/composite_HC_vs_HCB.mp4
 - 2-panel PNG: gqr_min_fig/Fig_mini_HC_vs_HCB.png
